# Basic regression with ARIMA errors.

This notebook contains an implemenation of regression with ARIMA errors.

In this implementation months of the year and weeks of the year are represented by seasonal indexes (dummy variables either 0 or 1).

## Imports

In [ ]:
import pandas as pd
import numpy as np

from forecast_tools.baseline import SNaive, Naive1
from forecast_tools.metrics import mean_absolute_error
from forecast_tools.datasets import load_emergency_dept

from pmdarima import auto_arima

import warnings
warnings.filterwarnings('ignore')

## Helper functions

In [ ]:
def preds_as_series(data, preds):
    '''
    Helper function for plotting predictions.
    Converts a numpy array of predictions to a 
    pandas.DataFrame with datetimeindex
    
    Parameters
    -----
    data - arraylike - the training data
    preds - numpy.array, vector of predictions 
    
    Returns:
    -------
    pandas.DataFrame
    '''
    start = pd.date_range(start=data.index.max(), periods=2, 
                          freq=data.index.freq).max()
    idx = pd.date_range(start=start, periods=len(preds), freq=data.index.freq)
    return pd.DataFrame(preds, index=idx)

In [ ]:
def plot_prediction_intervals(train, preds, intervals, 
                              test=None, show_train_size=None, figsize=(12,4)):
    '''
    Helper function to plot training data, point preds
    and 2 sets of prediction intevals
    
    assume 2 sets of PIs are provided!
    '''
    
    if show_train_size is None:
        show_train_size = len(train)
        
    train = train[-show_train_size:]
    ax = train.plot(figsize=figsize)
    

    mean = preds_as_series(train, preds)
    intervals_80 = preds_as_series(train, intervals[0])
    intervals_90 = preds_as_series(train, intervals[1])

    mean.plot(ax=ax, label='point forecast')

    ax.fill_between(intervals_80.index, mean[0], intervals_80[1], 
                    alpha=0.2,
                    label='80% PI', color='yellow');

    ax.fill_between(intervals_80.index,mean[0], intervals_80[0], 
                    alpha=0.2,
                    label='80% PI', color='yellow');

    ax.fill_between(intervals_80.index,intervals_80[1], intervals_90[1], 
                    alpha=0.2,
                    label='90% PI', color='purple');

    ax.fill_between(intervals_80.index,intervals_80[0], intervals_90[0], 
                    alpha=0.2,
                    label='90% PI', color='purple');
    
    if test is None:
        ax.legend(['train', 'point forecast', '80%PI', '_ignore','_ignore', 
                   '90%PI'], loc=2)
    else:
        test.plot(ax=ax, color='black', marker='o', ls='')
        ax.legend(['train', 'point forecast', 'Test', '80%PI', 
                   '_ignore','_ignore', '90%PI'], loc=2)

# Function to get seasonal indexes

In [ ]:
def get_seasonal_indexes(idx, include_month=True, include_dow=True):
    '''
    Seasonal indexes for use with regression.
    
    Params:
    ------
    idx: pd.DataTimeIndex
        Dates inclued in the dataframe
        
    include_month: bool, optional (default=True)
        Include 11 dummy variables for month of year
        
    include_dow: bool. optional (default=False)
        Include 6 dummy variables for month of year
        
    Returns:
    --------
    pd.DataFrame
    
    '''
    seasonal_idx = pd.DataFrame()
    
    if include_month:
        # uses the pd.get_dummies function 
        seasonal_idx = pd.concat([seasonal_idx, pd.get_dummies(idx.month,  prefix='m', 
                                                               drop_first=True)], axis=1)
        
    if include_dow:
        seasonal_idx = pd.concat([seasonal_idx, pd.get_dummies(idx.weekday, prefix='dow', 
                                                               drop_first=True)], axis=1)
        
    # set the index
    seasonal_idx.index = idx
        
    return seasonal_idx

## Example with `forecast_tools` ED dataset

In [ ]:
TARGET = 0.80
HOLDOUT = 28
PERIOD = 7

attends = load_emergency_dept()

# train-test split
train = attends[:-HOLDOUT]
test = attends[-HOLDOUT:]


X = get_seasonal_indexes(train.index)
# quick look at 
X.tail(7)

In [ ]:
model = auto_arima(train, exogenous=X, m=PERIOD, d=1, supress_warnings=True, maxiter=100)

In [ ]:
model.summary()

In [ ]:
def make_future_dataframe(h, y_train, include_mth=True, include_dow=True):
    '''
    Make a dataframe h steps into the future of y_train
    
    Params:
    ------
    h: int
        Forecast horizon
        
    y_train: pd.DataFrame
        Dataframe containing training data.  Must have a DataTimeIndex
    
    '''
    idx = pd.date_range(start=y_train.iloc[-1].name, periods=y_train.shape[0]+h, freq='D')
    seasonal_idxs = get_seasonal_indexes(idx, include_month=True, include_dow=True)
    return seasonal_idxs.iloc[-h:]


In [ ]:
future_dataframe = make_future_dataframe(HOLDOUT, train)
future_dataframe.head()

In [ ]:
def forecast(model, future_dataframe, return_predict_int=True, alpha=0.05):
    '''
    Forecast with regression with ARIMA errors
    
    Params:
    ------
    h: int
        Forecast horizon
        
    future_dataframe: pd.DataFrame
        Future dataframe containing datetimeindex + seasonal indexes
    
    return_predict_int: bool, optional (default=True)
        Prediction interval with predictions
        
    alpha: float, optional (Deault=0.05)
        1 - coverage for prediction interval
        
    Returns:
    --------
    preds, intervals
    '''
    
    h = future_dataframe.shape[0]
    return model.predict(n_periods=h, exogenous=future_dataframe, 
                         return_conf_int=return_predict_int, 
                         alpha=alpha)

In [ ]:
preds, intervals_95 = forecast(model, future_dataframe, alpha=0.05)
preds, intervals_80 = forecast(model, future_dataframe, alpha=0.2)
intervals = np.array([intervals_80, intervals_95])

In [ ]:
plot_prediction_intervals(train, preds, intervals, test=test, 
                          show_train_size=90)

In [33]:
import pandas as pd

from forecast_tools.model_selection import auto_naive

# the reattends data we used in class.
URL = "https://raw.githubusercontent.com/health-data-science-OR/hpdm097-datasets" \
    + "/master/ed_reattends_day.csv"

# PARAMETERS for analysis.
TRAIN_SIZE = len(reattends) // 3
HORIZON = 28
# seasonal period for auto_naive
PERIOD = 365
# step size for auto_naive (how many data points to add between splits.)
STEP = 7

# read data
reattends = pd.read_csv(URL, parse_dates=True, dayfirst=True, index_col='date')
reattends.index.freq = 'D'

# temporal train-test sploot
train, test = reattends[:TRAIN_SIZE], reattends[TRAIN_SIZE:]

# This is how big my training data set is (with default settings should be 516)
print(len(train))

# auto naive makes multiple train test splits with the training data.
# so we need the min training size to be > 365 

# just using rolling forecast origin
# I think the bug is with the method parameter.  if you set it to 'ro' it works.
results = auto_naive(train, horizon=HORIZON, step=STEP, seasonal_period=PERIOD, 
                   method='ro', metric='mae', min_train_size=365)

# mine selected average with a mae of around 22.6
print(results)

516
{'model': Average(), 'mae': 22.64866385564089}
